# Pregunta 16
Realizar una operación aritmética simple en columnas de un DataFrame.

In [35]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F 
from pyspark.sql import Window

spark = SparkSession.builder.appName("Pregunta 13").getOrCreate()

df = spark.read.csv("./datasets/employees.csv", header=True, inferSchema=True)

In [36]:
df.show(10)

+---+------------------+---+------+----------+------+------------+-----------------+----------+--------+-----------+-------+
| ID|              Name|Age|Gender|Department|Salary|Joining Date|Performance Score|Experience|  Status|   Location|Session|
+---+------------------+---+------+----------+------+------------+-----------------+----------+--------+-----------+-------+
|  1|      Cory Escobar| 48|Female|        HR|  5641|  2015-05-03|              2.0|        16|  Active|   New York|  Night|
|  2|   Timothy Sanchez| 25| Other|     Sales|  4249|  2020-11-09|              2.0|        11|Inactive|Los Angeles|Evening|
|  3|      Chad Nichols| 57| Other|     Sales|  3058|  2019-02-12|             NULL|         1|Inactive|   New York|Morning|
|  4|Christine Williams| 58|Female|        IT|  5895|  2017-09-08|              2.0|        13|Inactive|Los Angeles|Evening|
|  5|      Amber Harris| 35| Other|        IT|  4317|  2020-02-15|              5.0|        16|Inactive|   New York|Evening|


In [41]:
# primero obtenemos la columna de el tiempo total trabajado en años desde la fecha de contratacion hasta hoy (luego lo multiplicamos por 12 para saber los meses)
df = df.withColumn("Months_Worked", F.months_between(F.current_date(), F.col("Joining Date")).cast("int"))
df.show(10)

+---+------------------+---+------+----------+------+------------+-----------------+----------+--------+-----------+-------+-------------+
| ID|              Name|Age|Gender|Department|Salary|Joining Date|Performance Score|Experience|  Status|   Location|Session|Months_Worked|
+---+------------------+---+------+----------+------+------------+-----------------+----------+--------+-----------+-------+-------------+
|  1|      Cory Escobar| 48|Female|        HR|  5641|  2015-05-03|              2.0|        16|  Active|   New York|  Night|          125|
|  2|   Timothy Sanchez| 25| Other|     Sales|  4249|  2020-11-09|              2.0|        11|Inactive|Los Angeles|Evening|           59|
|  3|      Chad Nichols| 57| Other|     Sales|  3058|  2019-02-12|             NULL|         1|Inactive|   New York|Morning|           80|
|  4|Christine Williams| 58|Female|        IT|  5895|  2017-09-08|              2.0|        13|Inactive|Los Angeles|Evening|           97|
|  5|      Amber Harris| 35

In [46]:
# Obtenemos lo que le pago la empresa todo este tiempo
df = df.withColumn("Sueldo_total_recibido", F.col("Months_Worked") * F.col("Salary")).orderBy(F.col("Sueldo_total_recibido").desc())
df.show(20)

+---+------------------+---+------+----------+------+------------+-----------------+----------+--------+-----------+-------+-------------+---------------------+
| ID|              Name|Age|Gender|Department|Salary|Joining Date|Performance Score|Experience|  Status|   Location|Session|Months_Worked|Sueldo_total_recibido|
+---+------------------+---+------+----------+------+------------+-----------------+----------+--------+-----------+-------+-------------+---------------------+
|461|        Jose Allen| 27|Female|        HR|  9861|  2015-04-05|              3.0|        10|  Active|    Chicago|Morning|          126|              1242486|
|685|   Kristen Jackson| 50|Female|        IT|  9782|  2015-04-20|             NULL|         9|Inactive|    Chicago|  Night|          126|              1232532|
|380|         Dustin Le| 29|  Male|     Sales|  9726|  2015-04-13|              1.0|         2|Inactive|    Chicago|Evening|          126|              1225476|
|544|   Erin Cunningham| 63| Other